## Pulp Scheduling


In [6]:
from dataclasses import dataclass
from typing import Tuple, List
import pulp

### Classes

In [7]:
@dataclass
class Topic:
    name: str
    hours: float


@dataclass
class Course:
    name: str
    topics: List[Topic]
    final_day: int
    relative_proportion: float


class StudySpec:
    def __init__(self, courses: Tuple[Course], hours_per_day: List):
        self.validate_courses(courses, hours_per_day)
        self.courses = courses
        self.hours_per_day = hours_per_day

    def validate_courses(
        self, courses: Tuple[Course], hours_per_day: List[float]
    ) -> None:
        total_proportion = 0
        n_days = len(hours_per_day)

        for course in courses:
            total_proportion += course.relative_proportion

            if course.final_day > n_days:
                raise ValueError(
                    f"The final day of {course.name} ({course.final_day}) "
                    f"is outside the number of days ({n_days})."
                )

        if total_proportion != 1:
            raise ValueError(
                f"Relative proportion did not sum to 1 ({total_proportion})"
            )

    def rescale(self):
        study_hours = sum(self.hours_per_day)

        for i, course in enumerate(self.courses):
            course_target_hours = course.relative_proportion * study_hours
            rescaling_factor = course_target_hours / self.get_course_hours(i)

            # Rescale the associated topics
            self.scale_topic_hours(i, rescaling_factor)

        # return rescaling_factors

    def get_course_hours(self, index: int) -> float:
        total_hours = 0
        topics = self.courses[index].topics

        for topic in topics:
            total_hours += topic.hours

        return total_hours

    def scale_topic_hours(self, index: int, factor: float) -> None:
        topics = self.courses[index].topics

        for topic in topics:
            topic.hours *= factor
            topic.hours = round(2 * topic.hours) / 2 # Round to the nearest 0.5

### Functions

### Data Types

In [24]:
# Inputs
# 1)

# Maths
linear_algebra = Topic("Linear Algebra", 1)
geometry = Topic("Geometry", 2)
differential_equations = Topic("Differential Equations", 4)
maths = Course("Maths", [linear_algebra, geometry, differential_equations], 3, 0.3)

# Physics
o_and_w = Topic("Oscillations and Waves", 3)
mechanics = Topic("Mechanics", 3)
quantum_physics = Topic("Quantum Physics", 4)

physics = Course("Physics", [o_and_w, mechanics, quantum_physics], 5, 0.7)

# Creating the schedule
course_list = (maths, physics)
hours_per_day = [2, 3, 3, 0, 4, 3] # Total = 17
study_schedule = StudySpec(course_list, hours_per_day)

# 2)
# Output
topics_per_day = [
    [("Linear Algebra", 1), ("Geometry", 1)],
    [("Geometry", 1), ("Differential Equations", 2)],
    [("Oscillations and Waves", 3)],
    [("Differential Equations", 2)],
    [("Mechanics", 3)],
    [("Quantum Physics", 4)],
]

In [25]:
print(study_schedule.rescale())

None


In [26]:
print(study_schedule.courses)

(Course(name='Maths', topics=[Topic(name='Linear Algebra', hours=0.5), Topic(name='Geometry', hours=1.5), Topic(name='Differential Equations', hours=2.5)], final_day=3, relative_proportion=0.3), Course(name='Physics', topics=[Topic(name='Oscillations and Waves', hours=3.0), Topic(name='Mechanics', hours=3.0), Topic(name='Quantum Physics', hours=4.0)], final_day=5, relative_proportion=0.7))


In [27]:
print(study_schedule.get_course_hours(0))
print(study_schedule.get_course_hours(1))

4.5
10.0
